In [17]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from functions import custom_cv_eval, get_best_features, get_ev_from_df, get_bet_ev
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
np.set_printoptions(suppress=True)  #Suppress Scientific Notation

In [2]:
df=pd.read_csv("data/owl-with-odds.csv")
df_event=pd.read_csv("data/upcoming-with-odds.csv")

In [3]:
df.shape

(671, 58)

In [4]:
df_event.shape

(12, 58)

In [5]:
subset = ['t1_odds', 't2_odds']
df.dropna(subset=subset ,inplace=True)

df['t1_odds'] = pd.to_numeric(df['t1_odds'], errors='coerce')
df['t2_odds'] = pd.to_numeric(df['t2_odds'], errors='coerce')
df.dropna(subset=subset ,inplace=True)

In [6]:
subset = ['t1_odds', 't2_odds']
df_event.dropna(subset=subset ,inplace=True)

df_event['t1_odds'] = pd.to_numeric(df_event['t1_odds'], errors='coerce')
df_event['t2_odds'] = pd.to_numeric(df_event['t2_odds'], errors='coerce')
df_event.dropna(subset=subset ,inplace=True)

In [7]:
df_train=df.copy()
df_test = df_event.copy()

In [8]:
#inclusive features
features = ['team_one', 'team_two', 'corona_virus_isolation', 't1_wins_season',
       't1_losses_season', 't2_wins_season', 't2_losses_season',
       't1_matches_season', 't2_matches_season', 't1_win_percent_season',
       't2_win_percent_season', 't1_wins_alltime', 't1_losses_alltime',
       't2_wins_alltime', 't2_losses_alltime', 't1_matches_alltime',
       't2_matches_alltime', 't1_win_percent_alltime',
       't2_win_percent_alltime', 't1_wins_last_3', 't1_losses_last_3',
       't2_wins_last_3', 't2_losses_last_3', 't1_win_percent_last_3',
       't2_win_percent_last_3', 't1_wins_last_5', 't1_losses_last_5',
       't2_wins_last_5', 't2_losses_last_5', 't1_win_percent_last_5',
       't2_win_percent_last_5', 't1_wins_last_10', 't1_losses_last_10',
       't2_wins_last_10', 't2_losses_last_10', 't1_win_percent_last_10',
       't2_win_percent_last_10', 
        't1_wins_vs_t2', 't1_losses_vs_t2',
       't1_matches_vs_t2', 't1_odds', 't2_odds', 'winner_label']

In [9]:
df_test_filtered = df_test[features].copy()
df_train_filtered = df_train[features].copy()
df_test_filtered.dropna(inplace=True)
df_train_filtered.dropna(inplace=True)

df_test_team_names = df_test_filtered[['team_one', 'team_two']]
df_train_team_names = df_train_filtered[['team_one', 'team_two']]

In [10]:
model_5 = DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=12, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=4, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=75, splitter='best')
features_5 = ['t1_wins_last_3', 't2_matches_alltime', 'winner_label', 't1_odds', 't2_odds']
df_5 = df_train_filtered[features_5]

In [11]:
display(df_test_team_names)

,team_one,team_two
0,Shanghai Dragons,Seoul Dynasty
1,Chengdu Hunters,London Spitfire
2,Guangzhou Charge,New York Excelsior
3,Paris Eternal,Los Angeles Gladiators
5,Houston Outlaws,Atlanta Reign
6,Los Angeles Valiant,Boston Uprising
7,Hangzhou Spark,Shanghai Dragons
8,London Spitfire,Guangzhou Charge
9,Dallas Fuel,Philadelphia Fusion
10,Toronto Defiant,San Francisco Shock


In [12]:

X=df_train_filtered[features_5].copy()
X_test=df_test_filtered[features_5].copy()
y = X['winner_label']
y_test=X_test['winner_label']
X = X.drop('winner_label', axis=1)
X_test = X_test.drop('winner_label', axis=1)
model_5.fit(X, y)
probs=model_5.predict_proba(X_test)
X_test_list = np.array(X_test)
X_odds = list(zip(X_test_list[:, -2], X_test_list[:, -1], probs[:, 0], probs[:, 1], y_test))
ev_prepped_df = pd.DataFrame(X_odds, columns=['t1_odds', 't2_odds', 't1_prob', 't2_prob', 'winner'])
final_probs_df = pd.concat([ev_prepped_df, df_test_team_names.reset_index()], axis=1)
#display(ev_prepped_df)
#get_ev_from_df(ev_prepped_df, True, min_ev=99)
#probs_5 = probs

ev_prepped_df.shape

(10, 5)

In [13]:

display(final_probs_df)

,t1_odds,t2_odds,t1_prob,t2_prob,winner,index,team_one,team_two
0,-800.0,550.0,1.00,0.00,0,0,Shanghai Dragons,Seoul Dynasty
1,-185.0,160.0,0.25,0.75,0,1,Chengdu Hunters,London Spitfire
2,445.0,-590.0,0.75,0.25,0,2,Guangzhou Charge,New York Excelsior
3,145.0,-165.0,1.00,0.00,0,3,Paris Eternal,Los Angeles Gladiators
4,330.0,-410.0,0.00,1.00,0,5,Houston Outlaws,Atlanta Reign
5,-590.0,445.0,0.25,0.75,0,6,Los Angeles Valiant,Boston Uprising
6,560.0,-820.0,0.00,1.00,0,7,Hangzhou Spark,Shanghai Dragons
7,290.0,-350.0,0.75,0.25,0,8,London Spitfire,Guangzhou Charge
8,330.0,-410.0,0.00,1.00,0,9,Dallas Fuel,Philadelphia Fusion
9,600.0,-900.0,0.00,1.00,0,10,Toronto Defiant,San Francisco Shock


In [23]:
min_ev = 99

for index, row in final_probs_df.iterrows():
    team_one_ev = get_bet_ev(row['t1_odds'], row['t1_prob'])
    team_two_ev = get_bet_ev(row['t2_odds'], row['t2_prob'])
    if (team_one_ev >=99):
        print(f"BET: {row['team_one']} ({row['t1_odds']}) over {row['team_two']} ({row['t2_odds']}). EV: {team_one_ev}")
        
    if (team_two_ev >=99):
        print(f"BET: {row['team_two']} ({row['t2_odds']}) over {row['team_one']} ({row['t1_odds']}). EV: {team_two_ev}")
        

BET: Guangzhou Charge (445.0) over New York Excelsior (-590.0). EV: 308.75
BET: Paris Eternal (145.0) over Los Angeles Gladiators (-165.0). EV: 145.0
BET: Boston Uprising (445.0) over Los Angeles Valiant (-590.0). EV: 308.75
BET: London Spitfire (290.0) over Guangzhou Charge (-350.0). EV: 192.5
